In [2]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats as sp
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
A = np.array([12,20,39,17,23,32,21,23,10,9,17,24])
B = np.array([21,17,14,40,31,29,34,18,14,25,9,28])

In [11]:
AB = np.array([np.append(np.zeros(len(A)), np.ones(len(B))), np.append(A,B)])
AB


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [12., 20., 39., 17., 23., 32., 21., 23., 10.,  9., 17., 24., 21.,
        17., 14., 40., 31., 29., 34., 18., 14., 25.,  9., 28.]])

In [12]:
AB_t = AB.T
AB_t

array([[ 0., 12.],
       [ 0., 20.],
       [ 0., 39.],
       [ 0., 17.],
       [ 0., 23.],
       [ 0., 32.],
       [ 0., 21.],
       [ 0., 23.],
       [ 0., 10.],
       [ 0.,  9.],
       [ 0., 17.],
       [ 0., 24.],
       [ 1., 21.],
       [ 1., 17.],
       [ 1., 14.],
       [ 1., 40.],
       [ 1., 31.],
       [ 1., 29.],
       [ 1., 34.],
       [ 1., 18.],
       [ 1., 14.],
       [ 1., 25.],
       [ 1.,  9.],
       [ 1., 28.]])

A shuffled data with labels 
0 -> A
1 -> B

In [13]:
np.random.shuffle(AB_t)
AB_t

array([[ 0., 17.],
       [ 1., 14.],
       [ 0.,  9.],
       [ 0., 24.],
       [ 1., 21.],
       [ 0., 20.],
       [ 1., 28.],
       [ 1., 14.],
       [ 0., 10.],
       [ 0., 23.],
       [ 1., 34.],
       [ 0., 32.],
       [ 1.,  9.],
       [ 0., 39.],
       [ 1., 29.],
       [ 1., 31.],
       [ 1., 25.],
       [ 0., 23.],
       [ 1., 40.],
       [ 0., 12.],
       [ 1., 17.],
       [ 0., 17.],
       [ 0., 21.],
       [ 1., 18.]])

Create random lebels for the datas -> Chance

In [32]:
experimental_data = np.array([np.random.randint(0,2,len(AB_t)), AB_t[:, 1]]).T
experimental_data 

array([[ 1., 17.],
       [ 1., 14.],
       [ 0.,  9.],
       [ 1., 24.],
       [ 1., 21.],
       [ 1., 20.],
       [ 0., 28.],
       [ 0., 14.],
       [ 0., 10.],
       [ 1., 23.],
       [ 1., 34.],
       [ 0., 32.],
       [ 0.,  9.],
       [ 0., 39.],
       [ 1., 29.],
       [ 0., 31.],
       [ 1., 25.],
       [ 0., 23.],
       [ 1., 40.],
       [ 1., 12.],
       [ 1., 17.],
       [ 1., 17.],
       [ 0., 21.],
       [ 0., 18.]])

In [41]:
def AB_test(a:list,b:list,n:int):
    exp_diff_mean = np.empty([n, 1])
    ab = np.array([np.append(np.ones(len(a)), np.zeros(len(b))), np.append(a,b)]).T
    np.random.shuffle(ab)
    for i in range(n):
        exp_data = np.array([np.random.randint(0,2,len(ab)), ab[:,1]]).T
        exp_diff_mean[i] = exp_data[exp_data[:, 0] == 1][:1].mean() - exp_data[exp_data[:, 0] == 0][:,1].mean()
    return exp_diff_mean
    

In [42]:
AB_test(A,B,1000)

array([[-17.57692308],
       [ -7.        ],
       [-15.9       ],
       [-16.28571429],
       [ -5.81818182],
       [-16.3       ],
       [-18.5       ],
       [-11.58333333],
       [ -8.25      ],
       [-18.28571429],
       [-15.68181818],
       [-18.07142857],
       [-10.25      ],
       [ -7.23076923],
       [ -8.71428571],
       [-20.16666667],
       [-15.5       ],
       [ -7.13333333],
       [-20.86363636],
       [ -5.83333333],
       [-17.65384615],
       [ -9.4       ],
       [-15.73076923],
       [-13.9375    ],
       [-14.08333333],
       [-19.04545455],
       [ -7.38461538],
       [ -6.9       ],
       [ -8.5       ],
       [-18.5       ],
       [ -6.2       ],
       [-20.5       ],
       [ -9.8125    ],
       [-15.5       ],
       [-18.72222222],
       [-14.5       ],
       [ -7.5625    ],
       [-17.96153846],
       [ -8.92307692],
       [ -8.91666667],
       [-17.7       ],
       [-16.4       ],
       [-11.15384615],
       [ -8